In [65]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import pandas as pd

options=Options()
options.add_experimental_option('detach',True) #화면이 꺼지지 않고 유지
options.add_argument('--start-maximized') # 화면을 최대화
service=Service(ChromeDriverManager().install()) #크롬드라이버 자동 설치

driver=webdriver.Chrome(service=service, options=options)

start_area = '서울'
arrive_area = '제주'
start_day = '2024.10.12'
end_day = '2024.10.15'
member = {
    'adult':2,
    'youth':0,
    'baby':0
}
url = 'https://flight.naver.com/'

driver.get(url)
time.sleep(1)

# ✨ 출발지 설정
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[1]').click()
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[1]/div/input').send_keys(start_area)
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/section/div/a[1]').click()
time.sleep(1)

# ✨ 도착지 설정
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[1]/button[2]').click()
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[1]/div/input').send_keys(arrive_area)
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/section/div/a[1]').click()
time.sleep(1)

# ✨ 달력 조작부
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[2]/button[1]').click()
time.sleep(1)

# ✨ 출발 날짜 선택
s_calendar = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div')
s_find_month = s_calendar.find_element(By.XPATH,f"//div[text()='{start_day[:8]}']")
s_find_calendar = s_find_month.find_element(By.XPATH, ".//ancestor::div")
s_find_calendar.find_element(By.XPATH,f"//b[text()='{start_day[8:]}']").click()
time.sleep(1)

# ✨ 도착 날짜 선택
e_calendar = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[8]/div[2]/div[1]/div[2]/div')
e_find_month = e_calendar.find_element(By.XPATH,f"//div[text()='{end_day[:8]}']")
e_find_calendar = e_find_month.find_element(By.XPATH, ".//ancestor::div")
e_find_calendar.find_element(By.XPATH,f"//b[text()='{end_day[8:]}']").click()
time.sleep(1)

# ✨ 인원 선택
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[3]/button').click()
time.sleep(1)
if member['adult'] > 1:
    for i in range(2, member['adult']+1):
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/div/div/div[1]/div[1]/button[2]').click()
        time.sleep(1)
if member['youth'] > 0:
    for i in range(1, member['adult']+1):
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/div/div/div[1]/div[2]/button[2]').click()
        time.sleep(1)
if member['baby'] > 0:
    for i in range(1, member['adult']+1):
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/div[4]/div/div/div[1]/div[3]/button[2]').click()
        time.sleep(1)

# ✨ 검색시작
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/button[1]').click()
time.sleep(1)
driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[2]/div/div/div[2]/button[1]').click()
time.sleep(1)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.domestic_title__Gihg4')))

# ✨ 스크롤 내리기
before_location = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)
    after_location = driver.execute_script("return document.body.scrollHeight")
    if before_location == after_location:
        break
    before_location = after_location

driver.execute_script("window.scrollTo(0, 0)")
time.sleep(1)
go_list = driver.find_elements(By.CSS_SELECTOR,'div.domestic_Flight__8bR_b>div')

driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[4]/div/div[2]/div[2]/div/button').click()
time.sleep(1)

while True:
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)
    after_location = driver.execute_script("return document.body.scrollHeight")
    if before_location == after_location:
        break
    before_location = after_location

come_list = driver.find_elements(By.CSS_SELECTOR,'div.domestic_Flight__8bR_b>div')

go_res = []
for i in go_list:
    try:
        tmp = {}
        item = i.text.split('\n')
        tmp['항공사'] = item[0]
        tmp['출발시간'] = item[1]
        tmp['도착시간'] = item[2]
        tmp['운항시간'] = item[3]
        tmp['클래스'] = item[4].split()[0]
        tmp['가격'] = item[4].split()[1]
        go_res.append(tmp)
    except:
        pass

go_df = pd.DataFrame(go_res)
print(go_df.shape)
display(go_df.head())

come_res = []
for i in come_list:
    try:
        tmp = {}
        item = i.text.split('\n')
        tmp['항공사'] = item[0]
        tmp['출발시간'] = item[1]
        tmp['도착시간'] = item[2]
        tmp['운항시간'] = item[3]
        tmp['클래스'] = item[4].split()[0]
        tmp['가격'] = item[4].split()[1]
        come_res.append(tmp)
    except:
        pass

come_df = pd.DataFrame(come_res)
print(come_df.shape)
display(come_df.head())

(233, 6)


,항공사,출발시간,도착시간,운항시간,클래스,가격
0,아시아나항공,06:30CJU,07:40GMP,01시간 10분,비즈니스석편도,"154,700원~"
1,대한항공,07:00CJU,08:10GMP,01시간 10분,일반석편도,"94,700원~"
2,에어서울,07:45CJU,08:55GMP,01시간 10분,할인석편도,"54,700원~"
3,에어서울,07:45CJU,08:55GMP,01시간 10분,특가석편도,"57,100원~"
4,에어서울,07:45CJU,08:55GMP,01시간 10분,일반석편도,"82,800원~"


(233, 6)


,항공사,출발시간,도착시간,운항시간,클래스,가격
0,아시아나항공,06:30CJU,07:40GMP,01시간 10분,비즈니스석편도,"154,700원~"
1,대한항공,07:00CJU,08:10GMP,01시간 10분,일반석편도,"94,700원~"
2,에어서울,07:45CJU,08:55GMP,01시간 10분,할인석편도,"54,700원~"
3,에어서울,07:45CJU,08:55GMP,01시간 10분,특가석편도,"57,100원~"
4,에어서울,07:45CJU,08:55GMP,01시간 10분,일반석편도,"82,800원~"
